In [7]:
import pandas as pd
import datetime as dt

import plotly.express as px


df = pd.read_csv('PD_challange_data_set.csv')

def setUpTimeData(df):
    df['datetime'] = pd.to_datetime(df['time'])
    del df["time"]

    df['just_date'] = df['datetime'].dt.date
    df['just_time'] = df['datetime'].dt.time
    return df


df = setUpTimeData(df)
print(df.head())




   out_door_temp  electricity_usage            datetime   just_date just_time
0      33.630833             779.96 2016-12-30 01:30:00  2016-12-30  01:30:00
1      33.280833             804.26 2016-12-30 02:00:00  2016-12-30  02:00:00
2      33.003333             743.50 2016-12-30 02:30:00  2016-12-30  02:30:00
3      32.803333             703.86 2016-12-30 03:00:00  2016-12-30  03:00:00
4      32.555000             699.72 2016-12-30 03:30:00  2016-12-30  03:30:00


In [49]:
def sum_by_day(df):
    dfByDate = df.groupby(['just_date'], as_index=False).agg({'electricity_usage': ['sum'],'out_door_temp': ['mean']})
    dfByDate.reset_index()
    dfByDate.columns = dfByDate.columns.droplevel(1)   
    dfByDate['electricity_usage_sum_byDate'] = dfByDate['electricity_usage']
    del dfByDate['electricity_usage']
    dfByDate['out_door_temp_mean_byDate'] = dfByDate['out_door_temp']
    del dfByDate['out_door_temp']
    
    return dfByDate

def build_in_day_features(df):
#     datetime.strptime("25-01-1973", "%d-%m-%Y")
    df['just_date'] = pd.to_datetime(df['just_date'])
    df['day_num'] = df['just_date'].dt.weekday
    df['week_num'] = df['just_date'].dt.strftime("%V")
    df['year'] = df['just_date'].dt.strftime("%Y")
    df['yr_and_week_num'] = df['year'].map(str) + '-' + df['week_num'].map(str)
#     df['yr_and_week_num'] = str(df['year']) + str(df['week_num'])
    

    return df

DF_byDay = sum_by_day(df)
DF_byDay = build_in_day_features(DF_byDay)

print(DF_byDay.head())

   just_date  electricity_usage_sum_byDate  out_door_temp_mean_byDate  \
0 2016-12-30                      51800.48                  32.649648   
1 2016-12-31                      26829.30                  32.186580   
2 2017-01-01                      39417.32                  36.468906   
3 2017-01-02                      41160.22                  36.076892   
4 2017-01-03                      60663.12                  40.284983   

   day_num week_num  year yr_and_week_num  
0        4       52  2016         2016-52  
1        5       52  2016         2016-52  
2        6       52  2017         2017-52  
3        0       01  2017         2017-01  
4        1       01  2017         2017-01  


In [67]:
def organize_data_byWeek(df): 
    # remove first incomplete week and group by week
    weekDF = df.groupby('yr_and_week_num',as_index=False).agg({'electricity_usage_sum_byDate': ['sum', 'mean','std'],'out_door_temp_mean_byDate': ['mean'] })
    #format subheaders
    weekDF['electricity_usage_sum_byWeek'] = weekDF[(
        'electricity_usage_sum_byDate', 'sum')]
    weekDF['electricity_usage_mean_byWeek'] = weekDF[(
        'electricity_usage_sum_byDate', 'mean')]
    weekDF['electricity_usage_std_byWeek'] = weekDF[(
        'electricity_usage_sum_byDate', 'std')]
    weekDF['out_door_temp_mean_byWeek'] = weekDF[(
        'out_door_temp_mean_byDate', 'mean')]

    del weekDF["electricity_usage_sum_byDate"]
    del weekDF["out_door_temp_mean_byDate"]

    weekDF.reset_index()
    weekDF.columns = weekDF.columns.droplevel(1)   
    
    return weekDF

DF_byWeek = organize_data_byWeek(DF_byDay)

DF_byWeek.head()

,yr_and_week_num,electricity_usage_sum_byWeek,electricity_usage_mean_byWeek,electricity_usage_std_byWeek,out_door_temp_mean_byWeek
0,2016-52,78629.78,39314.890000,17657.290712,32.418114
1,2017-01,375624.18,53660.597143,9873.645809,29.387247
2,2017-02,398794.20,56970.600000,9162.547693,33.535836
3,2017-03,384035.28,54862.182857,9224.738864,38.697274
4,2017-04,392219.68,56031.382857,9847.583099,35.600893


In [74]:
def integrate_week_dateRef(week_data,day_data):    
    day_data_byMondays = day_data[day_data['day_num'] == 0]
    day_data_byMondays = day_data_byMondays.filter(['just_date','yr_and_week_num'])
    weekDF = pd.merge( day_data_byMondays, week_data, on='yr_and_week_num', how='inner')
    
    print(weekDF.head())

integrate_week_dateRef(DF_byWeek,DF_byDay)



   just_date yr_and_week_num  electricity_usage_sum_byWeek  \
0 2017-01-02         2017-01                     375624.18   
1 2017-01-09         2017-02                     398794.20   
2 2017-01-16         2017-03                     384035.28   
3 2017-01-23         2017-04                     392219.68   
4 2017-01-30         2017-05                     400551.64   

   electricity_usage_mean_byWeek  electricity_usage_std_byWeek  \
0                   53660.597143                   9873.645809   
1                   56970.600000                   9162.547693   
2                   54862.182857                   9224.738864   
3                   56031.382857                   9847.583099   
4                   57221.662857                   9151.186141   

   out_door_temp_mean_byWeek  
0                  29.387247  
1                  33.535836  
2                  38.697274  
3                  35.600893  
4                  28.863978  


In [94]:
# print(weekday_DF_byDay['electricity_usage_sum_bydate'].head())

def merge_week_and_day_data(DF_byDay,DF_byWeek):
    DF_byDay_with_weekdata = pd.merge(DF_byDay,DF_byWeek[10:], on='yr_and_week_num')

    def if_wknd(df):
        if df.day_num > 4:
            return True
        else:
            return False

    DF_byDay_with_weekdata['wknd'] = DF_byDay_with_weekdata.apply(if_wknd,axis=1)
    
    weekend_DF_byTime = DF_byDay_with_weekdata[DF_byDay_with_weekdata['wknd']==True] 
    wndmean = weekend_DF_byTime['electricity_usage_sum_byDate'].mean()

    print(wndmean,'mean')
    
    def if_holiday(df):
        if float(df.electricity_usage_sum_byDate) < wndmean + float(df.electricity_usage_std_byWeek) * 1 :
            return True
        else:
            return False
    
            
        
    DF_byDay_with_weekdata['wknd_or_hldy'] = DF_byDay_with_weekdata.apply(if_holiday,axis=1)

        
    #remove 
    return DF_byDay_with_weekdata

DF_byDay_with_weekdata = merge_week_and_day_data(DF_byDay, DF_byWeek)
weekday_DF_byTime = DF_byDay_with_weekdata[DF_byDay_with_weekdata['wknd_or_hldy']==False] 
weekend_DF_byTime = DF_byDay_with_weekdata[DF_byDay_with_weekdata['wknd_or_hldy']==True] 

print(len(weekend_DF_byTime),weekend_DF_byTime[30:].head(50)) 


43263.1443944 mean
197      just_date  electricity_usage_sum_byDate  out_door_temp_mean_byDate  \
98  2017-06-04                    38656.8800                  56.928420   
104 2017-06-10                    44769.5600                  64.546719   
105 2017-06-11                    39502.5000                  68.370903   
111 2017-06-17                    45385.8200                  66.930781   
112 2017-06-18                    40761.3200                  72.023437   
118 2017-06-24                    46934.8800                  70.163524   
119 2017-06-25                    47900.7000                  65.052778   
125 2017-07-01                    46394.4000                  72.520660   
126 2017-07-02                    40715.6000                  70.708958   
128 2017-07-04                    44919.4000                  66.432153   
132 2017-07-08                    46653.5200                  69.966024   
133 2017-07-09                    40672.6200                  63.361597   
13

In [106]:
weekday_DF_byTime = DF_byDay_with_weekdata[DF_byDay_with_weekdata['wknd_or_hldy']==False] 
weekend_DF_byTime = DF_byDay_with_weekdata[DF_byDay_with_weekdata['wknd_or_hldy']==True] 



In [70]:
# def sum_by_day(df):
#     dfByDate = df.groupby('just_date').agg({'electricity_usage': ['sum'],'out_door_temp': ['mean'],'electricity_usage_std_byWeek':['mean']})
#     dfByDate['electricity_usage_sum_bydate'] = dfByDate[('electricity_usage', 'sum')]
#     dfByDate['out_door_temp_mean_bydate'] = dfByDate[('out_door_temp', 'mean')]
#     dfByDate['electricity_usage_std_byWeek_agg'] = dfByDate[('electricity_usage_std_byWeek', 'mean')]
#     del dfByDate["electricity_usage"]
#     del dfByDate["out_door_temp"]
#     del dfByDate['electricity_usage_std_byWeek']
    
#     return dfByDate

# weekday_DF_byDay = sum_by_day(weekday_DF_byTime)
# weekend_DF_byDay = sum_by_day(weekend_DF_byTime)

homevest 

# print(weekday_DF_byDay.head())


KeyError: 'electricity_usage'